In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

In [3]:
def clear_trash():
  for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
    # print the name of the file being deleted.
    try:
      print(f"the file {a_file['title']}, is about to get deleted permanently.")
      # delete the file permanently.
      a_file.Delete()
    except:
      pass

In [4]:
# clear_trash()

In [5]:
# !cp /content/drive/MyDrive/TalkLikeMom/conversations-main.zip /content
# !unzip conversations-main.zip

In [6]:
cd /content/conversations-main/Classifier/

/content/conversations-main/Classifier


In [7]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [8]:
import torch
torch.cuda.is_available()

True

In [9]:
# !pip uninstall --yes torchaudio torchdata torchtext torchvision

In [10]:
# !pip install -q torch==1.13.0

In [11]:
torch.__version__

'2.0.1+cu118'

In [12]:
import sys
sys.path.append('..')

In [13]:
from utils import load_jsonl, dump_jsonl, set_random_seed

In [14]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

In [15]:
import pandas as pd

In [16]:
# df = get_task1_conver("../Task1//annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=False)
# # print(df[0]["text"][0])
# pd.concat(df).groupby("label").count()

In [17]:
# df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# # print(df[0]["text"][0])
# pd.concat(df).groupby("label").count()

In [18]:
# df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# # print(df[0]["text"][0])
# pd.concat(df).groupby("label").count()

In [19]:
# "DONE"

In [20]:
# !pip install -q transformers pythainlp datasets evaluate sentencepiece
# # !pip install -q accelerate -U
# !pip install -q transformers[torch]
# # !pip install -q
# !pip install -q shap nlp

In [21]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import wandb
# from sklearn.metrics import precision_recall_fscore_support, classification_report
# from pythainlp.tokenize import word_tokenize
import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer



In [22]:
import random
def set_random_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    np.random.RandomState(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) #seed all gpus
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False

In [60]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
import os, shutil
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score

def run_exp(out_dir, df, seed=42, report="none", regressor_configs=None):

    set_random_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    # device = torch.device("cpu")
    print("START")
    print("step 1: load data")
    train, val, test = df

#     train = train.head(100)
#     val = val.head(100)
#     test = test.head(100)

    print("step 2: load tokenizer")
    model_name = "airesearch/wangchanberta-base-att-spm-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

    print("step 3: init data")
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train)
    ds['val'] = Dataset.from_pandas(val)
    ds['test'] = Dataset.from_pandas(test)

    if regressor_configs is None:
        labels = train["label"].unique()
        num_labels = len(labels)
        print(labels)

        class_weights = compute_class_weight("balanced", classes=labels, y=train["label"].values)
        class_weights = torch.tensor(class_weights).float().to(device)

        id2label = {i:l for i, l in enumerate(labels)}
        label2id = {l:i for i, l in enumerate(labels)}

        def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
            texts = [preprocess(t) for t in d["text"]]
    #         print(texts)
            tokens = tokenizer(texts, truncation=True, max_length=max_length)
            num = [len(t) for t in tokens["input_ids"]]
    #         print(num)
    #         print("AVG", len(num), sum(num)/len(num))
            tokens["label"] = [label2id[label] for label in d["label"]]
            return tokens
    else:
#         labels = train["label"].unique()
        num_labels = 1

        id2label = {1: regressor_configs["label"]}
        label2id = [regressor_configs["label"]]

        def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
            texts = [preprocess(t) for t in d["text"]]
    #         print(texts)
            tokens = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length)
            # num = [len(t) for t in tokens["input_ids"]]
    #         print(num)
    #         print("AVG", len(num), sum(num)/len(num))
            tokens["label"] = [regressor_configs["label_fn"](label) for label in d["label"]]
            return tokens



    tokenized_ds = ds.map(word_tokenize, batched=True, fn_kwargs={"tokenizer":tokenizer, "label2id": label2id, "max_length":max_length})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    print("step 4: load model")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id);
    model.resize_token_embeddings(len(tokenizer)+1)
    model = model.to(device)

    # print(model)

    if regressor_configs is None:
        metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)
            return metrics.compute(predictions=predictions, references=labels, average="macro")
    else:
        label_fn = regressor_configs["label_fn"]
        def compute_metrics(eval_pred):
            predictions, actual = eval_pred
            predictions = predictions.reshape(-1)

            predicted_labels = [label_fn(p) for p in predictions]
            actual_labels = [label_fn(p) for p in actual]
            p, r, f1, _ = precision_recall_fscore_support(actual_labels, predicted_labels, average='macro')

            return {
                "r2_score": r2_score(actual, predictions),
                "mean_squared_error": np.sqrt(mean_squared_error(actual, predictions)),
                "accuracy": accuracy_score(actual_labels, predicted_labels),
                "f1": f1,
                "precision": p,
                "recall": r,
            }


    print("step 5: fine-tune")

    training_args = TrainingArguments(
        output_dir=out_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to=report,
        metric_for_best_model="f1",
        save_total_limit=2,
        load_best_model_at_end=True,
        push_to_hub=False,
        run_name=out_dir,
    )

    # print(training_args)

    if regressor_configs is None:
        class CustomTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                # forward pass
                outputs = model(**inputs)

                logits = outputs.get("logits")

                loss_fct = nn.CrossEntropyLoss(weight=class_weights)
                loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                return (loss, outputs) if return_outputs else loss
    else:
        CustomTrainer = Trainer

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    best_ckpt_path = trainer.state.best_model_checkpoint
    print(best_ckpt_path)

    modle_out_path = out_dir+"/best_model"
    if os.path.exists(modle_out_path):
        shutil.rmtree(modle_out_path)

    os.rename(best_ckpt_path, modle_out_path)
    best_ckpt_path = modle_out_path

    print("step 6: evaluate")
    eval_test = trainer.evaluate(tokenized_ds["test"])
    eval_val = trainer.evaluate(tokenized_ds["val"])
    print(eval_test)

    print("DONE")

    return trainer, eval_val, eval_test

## Task1: Train Model

In [61]:
report = "none"
batch_size = 16
max_length = 128
num_epochs = 20

In [62]:
# import os
# stream = os.popen('nohup python3 run_train_task_classifier.py > train2.out &')
# output = stream.read()
# output

In [63]:
# df[0]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [64]:
# df[1]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [65]:
# df[2]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [67]:
def closeness_label_fn(label):
    if label == '1. Close':
        return 1
    elif label =='2. Know each other':
        return 0.5
    elif label == "3. Don't know each other":
        return 0
    elif type(label)==str:
        assert(False)

    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>

    if label > 0.66:
        return '1. Close'
    elif label > 0.33:
        return '2. Know each other'
    else:
        return "3. Don't know each other"


best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task1_clse_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1096 60 60
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.138625,0.342276,0.372323,0.550000,0.518770,0.573650,0.524442
2,No log,0.252261,-0.196891,0.502256,0.266667,0.235317,0.343569,0.248505
3,No log,0.211950,-0.005627,0.460380,0.183333,0.152778,0.548485,0.334599
4,No log,0.219781,-0.042781,0.468808,0.183333,0.141197,0.386905,0.359731
5,No log,0.209669,0.005194,0.457896,0.183333,0.142886,0.385057,0.362319
6,No log,0.272995,-0.295265,0.522489,0.383333,0.184739,0.127778,0.333333
7,No log,0.237655,-0.127588,0.487499,0.166667,0.116013,0.384181,0.347826
8,0.207400,0.254820,-0.209031,0.504797,0.383333,0.191667,0.134503,0.333333
9,0.207400,0.277451,-0.316407,0.526736,0.383333,0.184739,0.127778,0.333333
10,0.207400,0.236625,-0.122702,0.486441,0.183333,0.142886,0.385057,0.362319


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted s

./Regressors/task1_clse_usr4/checkpoint-69
step 6: evaluate


{'eval_loss': 0.10695171356201172, 'eval_r2_score': 0.4759766765699701, 'eval_mean_squared_error': 0.32703471183776855, 'eval_accuracy': 0.6333333333333333, 'eval_f1': 0.6041083099906629, 'eval_precision': 0.6555555555555556, 'eval_recall': 0.616905615292712, 'eval_runtime': 0.154, 'eval_samples_per_second': 389.652, 'eval_steps_per_second': 25.977, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.10695171356201172, 'eval_r2_score': 0.4759766765699701, 'eval_mean_squared_error': 0.32703471183776855, 'eval_accuracy': 0.6333333333333333, 'eval_f1': 0.6041083099906629, 'eval_precision': 0.6555555555555556, 'eval_recall': 0.616905615292712, 'eval_runtime': 0.154, 'eval_samples_per_second': 389.652, 'eval_steps_per_second': 25.977, 'epoch': 20.0}
Result[test] {'eval_loss': 0.13862453401088715, 'eval_r2_score': 0.34227571271991475, 'eval_mean_squared_error': 0.3723231256008148, 'eval_accuracy': 0.55, 'eval_f1': 0.5187700433880399, 'eval_precision': 0.5736495388669302, 'eval_

In [45]:
# !zip -r task1.zip Regressors/task1_*

In [46]:
# !cp -r Regressors/task1_* /content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors

In [68]:
# !mkdir ShapleyValuesV2

In [69]:
import shap
import scipy as sp

import pickle

def load_shap_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

def save_shap_values(filepath, obj):
    with open(filepath, 'wb') as fin:
        pickle.dump(obj, fin)


def get_shapley(df, trainer, shap_path):
    train, val, test = df
    train["split"] = "train"
    val["split"] = "val"
    test["split"] = "test"

    df = pd.concat([train, test, val])
    print("DATA SIZE", len(df))
    def f(x):
        tokenizer = trainer.tokenizer
        model = trainer.model

        # print(type(x.tolist()))
        tv = tokenizer(x.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
        input_ids = tv["input_ids"].cuda()
        attention_mask = tv["attention_mask"].cuda()
        # tv = tokenizer(x, truncation=True, max_length=max_length).cuda()
        outputs = model(input_ids, attention_mask=attention_mask)[0].detach().cpu().numpy()

        # scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        # val = sp.special.logit(scores[:,1]) # use one vs rest logit units

        val = outputs.reshape(-1)
        return val

    explainer = shap.Explainer(f, trainer.tokenizer)
    shap_values = explainer({
        "label": df["label"].values,
        "text": [preprocess(t) for t in df["text"].values],
    })

    save_shap_values(shap_path, shap_values)

    return shap_values

get_shapley(df, best_trainer, "./ShapleyValuesV2/task1_clse_regressor.pkl");

DATA SIZE 1216


PartitionExplainer explainer: 1217it [38:08,  1.89s/it]


In [70]:
cp ./ShapleyValuesV2/task1_clse_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV2/task1_clse_regressor.pkl

In [91]:
def authority1_label_fn(label):
    if label == '0. Very respect':
        return 1
    elif label =='1. Respect':
        return 0.5
    elif label == "2. Normal":
        return 0
    elif type(label)==str:
        assert(False)

    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>

    if label > 0.66:
        return '0. Very respect'
    elif label > 0.33:
        return '1. Respect'
    else:
        return "2. Normal"

In [92]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task1_auth_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority1_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1098 61 61
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1098 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.156547,-0.003463,0.395660,0.508197,0.326349,0.294715,0.366013
2,No log,0.173132,-0.109776,0.416092,0.508197,0.234848,0.191358,0.303922
3,No log,0.133511,0.144198,0.365391,0.475410,0.405749,0.527778,0.407190
4,No log,0.121028,0.224212,0.347891,0.590164,0.522876,0.578431,0.511765
5,No log,0.128388,0.177036,0.358312,0.622951,0.608370,0.650285,0.616993
6,No log,0.136734,0.123533,0.369776,0.590164,0.486869,0.661499,0.468954
7,No log,0.124425,0.202439,0.352739,0.557377,0.486722,0.550694,0.486601
8,0.140200,0.125406,0.196148,0.354127,0.590164,0.511111,0.553333,0.518627
9,0.140200,0.121857,0.218901,0.349080,0.704918,0.623529,0.701852,0.617647
10,0.140200,0.131749,0.155488,0.362973,0.639344,0.534942,0.635714,0.523203


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


./Regressors/task1_auth_usr4/checkpoint-621
step 6: evaluate


{'eval_loss': 0.2092660665512085, 'eval_r2_score': -0.20819062150641, 'eval_mean_squared_error': 0.4574560225009918, 'eval_accuracy': 0.5081967213114754, 'eval_f1': 0.45588235294117646, 'eval_precision': 0.4837962962962963, 'eval_recall': 0.4489087301587302, 'eval_runtime': 0.1587, 'eval_samples_per_second': 384.288, 'eval_steps_per_second': 25.199, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.2092660665512085, 'eval_r2_score': -0.20819062150641, 'eval_mean_squared_error': 0.4574560225009918, 'eval_accuracy': 0.5081967213114754, 'eval_f1': 0.45588235294117646, 'eval_precision': 0.4837962962962963, 'eval_recall': 0.4489087301587302, 'eval_runtime': 0.1587, 'eval_samples_per_second': 384.288, 'eval_steps_per_second': 25.199, 'epoch': 20.0}
Result[test] {'eval_loss': 0.12185657024383545, 'eval_r2_score': 0.21890051071793581, 'eval_mean_squared_error': 0.34907957911491394, 'eval_accuracy': 0.7049180327868853, 'eval_f1': 0.6235294117647059, 'eval_precision': 0.701851851

In [93]:
get_shapley(df, best_trainer, "./ShapleyValuesV2/task1_auth_regressor.pkl");

DATA SIZE 1220


PartitionExplainer explainer: 1221it [38:23,  1.90s/it]


In [94]:
cp ./ShapleyValuesV2/task1_auth_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV2/task1_auth_regressor.pkl

## Task2: Train Model

In [78]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", seed=i, skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task2_clse_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1495 186 186
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1495 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

step 4: load model


...

step 5: fine-tune


You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.103519,0.104216,0.321744,0.725806,0.402179,0.517516,0.425800
2,No log,0.091967,0.204178,0.303261,0.688172,0.374359,0.417371,0.395244
3,No log,0.093850,0.187889,0.306349,0.725806,0.508274,0.608937,0.504335
4,No log,0.092707,0.197780,0.304478,0.741935,0.483328,0.588647,0.469739
5,No log,0.090140,0.219993,0.300233,0.736559,0.446706,0.494025,0.440236
6,0.112800,0.081157,0.297722,0.284881,0.790323,0.551066,0.607942,0.530598
7,0.112800,0.090962,0.212872,0.301600,0.774194,0.476119,0.627282,0.456439
8,0.112800,0.098495,0.147694,0.313839,0.801075,0.530198,0.672090,0.493687
9,0.112800,0.085990,0.255901,0.293240,0.811828,0.565477,0.676543,0.525505
10,0.112800,0.093868,0.187727,0.306379,0.795699,0.486656,0.557437,0.462668


./Regressors/task2_clse_usr4/checkpoint-846
step 6: evaluate


{'eval_loss': 0.09288256615400314, 'eval_r2_score': 0.16017895090620626, 'eval_mean_squared_error': 0.3047664165496826, 'eval_accuracy': 0.7688172043010753, 'eval_f1': 0.403699540112417, 'eval_precision': 0.5, 'eval_recall': 0.40138037196860726, 'eval_runtime': 0.4876, 'eval_samples_per_second': 381.462, 'eval_steps_per_second': 24.61, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.09288256615400314, 'eval_r2_score': 0.16017895090620626, 'eval_mean_squared_error': 0.3047664165496826, 'eval_accuracy': 0.7688172043010753, 'eval_f1': 0.403699540112417, 'eval_precision': 0.5, 'eval_recall': 0.40138037196860726, 'eval_runtime': 0.4876, 'eval_samples_per_second': 381.462, 'eval_steps_per_second': 24.61, 'epoch': 20.0}
Result[test] {'eval_loss': 0.08598995208740234, 'eval_r2_score': 0.25590090592993253, 'eval_mean_squared_error': 0.29324042797088623, 'eval_accuracy': 0.8118279569892473, 'eval_f1': 0.5654769434655583, 'eval_precision': 0.6765432098765433, 'eval_recall': 0.52

In [80]:
report = "none"
batch_size = 16
max_length = 128
num_epochs = 100

In [82]:
df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", seed=42, skips = ["4. Don't like each other"], only_user=True)
trainer, eval_test, eval_val = run_exp(f"./Regressors/task2_clse_usr{i}", df, seed=42, report=report, regressor_configs={
    "label": "close",
    "not_label": "not_close",
    "label_fn": closeness_label_fn,
})

print("Result[val]", eval_val)
print("Result[test]", eval_test)

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1495 186 186
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1495 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.114168,0.093053,0.337888,0.618280,0.431471,0.674068,0.458771
2,No log,0.120355,0.043900,0.346923,0.747312,0.312443,0.294697,0.342908
3,No log,0.088404,0.297725,0.297327,0.731183,0.481485,0.666948,0.458842
4,No log,0.087330,0.306251,0.295517,0.736559,0.449241,0.686036,0.456903
5,No log,0.112012,0.110176,0.334683,0.752688,0.434903,0.657949,0.417754
6,0.106600,0.085138,0.323665,0.291784,0.768817,0.547919,0.709732,0.529267
7,0.106600,0.086346,0.314067,0.293848,0.736559,0.505930,0.637330,0.486478
8,0.106600,0.094282,0.251026,0.307054,0.774194,0.453008,0.694785,0.444846
9,0.106600,0.084130,0.331673,0.290052,0.752688,0.492542,0.638983,0.485934
10,0.106600,0.101224,0.195878,0.318157,0.779570,0.485453,0.643155,0.472482


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


./Regressors/task2_clse_usr4/checkpoint-4324
step 6: evaluate


{'eval_loss': 0.1276169717311859, 'eval_r2_score': -0.013727498563620877, 'eval_mean_squared_error': 0.3572351634502411, 'eval_accuracy': 0.7634408602150538, 'eval_f1': 0.39485936832790564, 'eval_precision': 0.4395663956639566, 'eval_recall': 0.3880727773345223, 'eval_runtime': 0.4885, 'eval_samples_per_second': 380.795, 'eval_steps_per_second': 24.567, 'epoch': 100.0}
DONE
Result[val] {'eval_loss': 0.1276169717311859, 'eval_r2_score': -0.013727498563620877, 'eval_mean_squared_error': 0.3572351634502411, 'eval_accuracy': 0.7634408602150538, 'eval_f1': 0.39485936832790564, 'eval_precision': 0.4395663956639566, 'eval_recall': 0.3880727773345223, 'eval_runtime': 0.4885, 'eval_samples_per_second': 380.795, 'eval_steps_per_second': 24.567, 'epoch': 100.0}
Result[test] {'eval_loss': 0.08259650319814682, 'eval_r2_score': 0.34385560339569476, 'eval_mean_squared_error': 0.28739607334136963, 'eval_accuracy': 0.7795698924731183, 'eval_f1': 0.5721034839214372, 'eval_precision': 0.6136111111111111,

In [83]:
report = "none"
batch_size = 16
max_length = 128
num_epochs = 20

In [84]:
get_shapley(df, best_trainer, "./ShapleyValuesV2/task2_clse_regressor.pkl");

DATA SIZE 1867


PartitionExplainer explainer: 1868it [41:25,  1.34s/it]


In [85]:
cp ./ShapleyValuesV2/task2_clse_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV2/task2_clse_regressor.pkl

In [86]:
def authority2_label_fn(label):
    if label == '1. Respect':
        return 1
    elif label =='2. Normal':
        return 0.5
    elif label == "3. Not respect":
        return 0
    elif type(label)==str:
        assert(False)

    if label > 0.66:
        return '1. Respect'
    elif label > 0.33:
        return '2. Normal'
    else:
        return "3. Not respect"

In [88]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", seed=i, skips = [], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task2_auth_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1876 234 234
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1876 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

step 4: load model


...

step 5: fine-tune


You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.050687,0.393914,0.225138,0.735043,0.683315,0.681266,0.686472
2,No log,0.040865,0.511363,0.202150,0.764957,0.704537,0.723504,0.730473
3,No log,0.056501,0.324393,0.237700,0.675214,0.639127,0.650983,0.713646
4,No log,0.042851,0.487609,0.207006,0.696581,0.707016,0.739010,0.745183
5,0.066400,0.035469,0.575888,0.188331,0.794872,0.764414,0.742192,0.793631
6,0.066400,0.035130,0.579933,0.187431,0.807692,0.783992,0.794841,0.781327
7,0.066400,0.039281,0.530304,0.198194,0.756410,0.723291,0.694079,0.782226
8,0.066400,0.037386,0.552955,0.193356,0.747863,0.741656,0.739884,0.769361
9,0.032200,0.035372,0.577043,0.188074,0.794872,0.752811,0.737672,0.780584
10,0.032200,0.040446,0.516367,0.201113,0.786325,0.735651,0.757681,0.758754


./Regressors/task2_auth_usr4/checkpoint-708
step 6: evaluate


{'eval_loss': 0.03569203242659569, 'eval_r2_score': 0.571319862367159, 'eval_mean_squared_error': 0.18892334401607513, 'eval_accuracy': 0.7692307692307693, 'eval_f1': 0.7466489832007074, 'eval_precision': 0.7442687890449085, 'eval_recall': 0.7816906963248428, 'eval_runtime': 0.5726, 'eval_samples_per_second': 408.655, 'eval_steps_per_second': 26.196, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.03569203242659569, 'eval_r2_score': 0.571319862367159, 'eval_mean_squared_error': 0.18892334401607513, 'eval_accuracy': 0.7692307692307693, 'eval_f1': 0.7466489832007074, 'eval_precision': 0.7442687890449085, 'eval_recall': 0.7816906963248428, 'eval_runtime': 0.5726, 'eval_samples_per_second': 408.655, 'eval_steps_per_second': 26.196, 'epoch': 20.0}
Result[test] {'eval_loss': 0.03513020649552345, 'eval_r2_score': 0.5799334888197065, 'eval_mean_squared_error': 0.18743053078651428, 'eval_accuracy': 0.8076923076923077, 'eval_f1': 0.7839922854387656, 'eval_precision': 0.79484126

In [89]:
get_shapley(df, best_trainer, "./ShapleyValuesV2/task2_auth_regressor.pkl");

DATA SIZE 2344


PartitionExplainer explainer: 2345it [53:15,  1.37s/it]


In [90]:
cp ./ShapleyValuesV2/task2_auth_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV2/task2_auth_regressor.pkl

## Task3: Train Model

In [72]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task3_clse_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1090 60 60
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1090 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.079497,0.042850,0.281951,0.616667,0.539683,0.652137,0.556111
2,No log,0.060752,0.268537,0.246479,0.716667,0.489744,0.516667,0.517917
3,No log,0.042141,0.492622,0.205282,0.783333,0.695560,0.856867,0.657361
4,No log,0.073381,0.116486,0.270889,0.816667,0.563301,0.552855,0.574583
5,No log,0.039103,0.529199,0.197744,0.800000,0.678788,0.707937,0.667778
6,No log,0.038289,0.538995,0.195676,0.800000,0.547563,0.532738,0.564167
7,No log,0.038984,0.530628,0.197443,0.800000,0.547605,0.533333,0.567083
8,0.059100,0.061923,0.254443,0.248843,0.683333,0.590423,0.677579,0.600694
9,0.059100,0.129602,-0.560431,0.360003,0.516667,0.425568,0.434480,0.541806
10,0.059100,0.062767,0.244281,0.250533,0.716667,0.601471,0.601669,0.604028


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


./Regressors/task3_clse_usr4/checkpoint-897
step 6: evaluate


{'eval_loss': 0.05288446322083473, 'eval_r2_score': 0.3019832575886975, 'eval_mean_squared_error': 0.2299662083387375, 'eval_accuracy': 0.75, 'eval_f1': 0.6370370370370372, 'eval_precision': 0.6628282828282829, 'eval_recall': 0.63003003003003, 'eval_runtime': 0.1557, 'eval_samples_per_second': 385.378, 'eval_steps_per_second': 25.692, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.05288446322083473, 'eval_r2_score': 0.3019832575886975, 'eval_mean_squared_error': 0.2299662083387375, 'eval_accuracy': 0.75, 'eval_f1': 0.6370370370370372, 'eval_precision': 0.6628282828282829, 'eval_recall': 0.63003003003003, 'eval_runtime': 0.1557, 'eval_samples_per_second': 385.378, 'eval_steps_per_second': 25.692, 'epoch': 20.0}
Result[test] {'eval_loss': 0.04255913197994232, 'eval_r2_score': 0.487582401406467, 'eval_mean_squared_error': 0.20629864931106567, 'eval_accuracy': 0.85, 'eval_f1': 0.7135572139303482, 'eval_precision': 0.7472049689440995, 'eval_recall': 0.6931944444444444, 'e

In [73]:
get_shapley(df, best_trainer, "./ShapleyValuesV2/task3_clse_regressor.pkl");

DATA SIZE 1210


PartitionExplainer explainer: 1211it [38:13,  1.90s/it]


In [74]:
cp ./ShapleyValuesV2/task3_clse_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV2/task3_clse_regressor.pkl


In [95]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task3_auth_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1099 61 61
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1099 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.048106,-0.181533,0.219331,0.868852,0.659164,0.674359,0.647059
2,No log,0.034104,0.162358,0.184674,0.819672,0.630363,0.647222,0.627451
3,No log,0.043780,-0.075277,0.209236,0.803279,0.617143,0.632653,0.620915
4,No log,0.034721,0.147220,0.186335,0.836066,0.644976,0.665733,0.633987
5,No log,0.055884,-0.372558,0.236397,0.508197,0.304386,0.350934,0.447712
6,No log,0.066454,-0.632192,0.257788,0.524590,0.494241,0.584628,0.607843
7,No log,0.042695,-0.048627,0.206627,0.770492,0.590247,0.609903,0.607843
8,0.041100,0.039970,0.018287,0.199926,0.737705,0.393939,0.375000,0.441176
9,0.041100,0.034684,0.148136,0.186235,0.836066,0.644976,0.665733,0.633987
10,0.041100,0.035661,0.124134,0.188841,0.852459,0.652692,0.667949,0.640523


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


./Regressors/task3_auth_usr4/checkpoint-69
step 6: evaluate


{'eval_loss': 0.028756296262145042, 'eval_r2_score': -0.06469832403861187, 'eval_mean_squared_error': 0.16957682371139526, 'eval_accuracy': 0.8360655737704918, 'eval_f1': 0.43580246913580245, 'eval_precision': 0.46913580246913583, 'eval_recall': 0.41358024691358025, 'eval_runtime': 0.1531, 'eval_samples_per_second': 398.38, 'eval_steps_per_second': 26.123, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.028756296262145042, 'eval_r2_score': -0.06469832403861187, 'eval_mean_squared_error': 0.16957682371139526, 'eval_accuracy': 0.8360655737704918, 'eval_f1': 0.43580246913580245, 'eval_precision': 0.46913580246913583, 'eval_recall': 0.41358024691358025, 'eval_runtime': 0.1531, 'eval_samples_per_second': 398.38, 'eval_steps_per_second': 26.123, 'epoch': 20.0}
Result[test] {'eval_loss': 0.04810596629977226, 'eval_r2_score': -0.18153345402199905, 'eval_mean_squared_error': 0.21933074295520782, 'eval_accuracy': 0.8688524590163934, 'eval_f1': 0.6591644601353339, 'eval_precisio

In [96]:
get_shapley(df, best_trainer, "./ShapleyValuesV2/task3_auth_regressor.pkl");

DATA SIZE 1221


PartitionExplainer explainer: 1222it [39:09,  1.93s/it]


In [97]:
cp ./ShapleyValuesV2/task3_auth_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV2/task3_auth_regressor.pkl

In [98]:
cp -r ./Regressors /content/drive/MyDrive/TalkLikeMom/src/Classifier/RegressorsV2

sample_data/
